In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]
NUM_EPOCHS = 30
LEARNING_RATE = 1e-3

In [3]:
train_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

In [4]:
train_dataset_path = '../../../dataset/train'
val_dataset_path = '../../../dataset/val'

train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dataset_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [5]:
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_FEATURES)
features = model.features
features.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg16_features-amdegroot-88682ab5.pth" to C:\Users\Raiza Rahman/.cache\torch\hub\checkpoints\vgg16_features-amdegroot-88682ab5.pth
100%|██████████| 528M/528M [01:17<00:00, 7.16MB/s] 


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [6]:
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(4096, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)

In [8]:
def train_one_epoch(model, device, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return running_loss/total, correct/total

In [9]:
def validate(model, device, loader, criterion):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return val_loss/total, correct/total

In [10]:
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, device, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, device, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f}, acc {train_acc:.4f}"
          )
    
    if(val_loss < best_val_loss):
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_vgg16.pth')

Total batches: 290
Epoch 1/30: Train loss 1.7899, acc 0.4658 | Val   loss 1.1541, acc 0.6313
Epoch 2/30: Train loss 1.3641, acc 0.6366 | Val   loss 0.9238, acc 0.7121
Epoch 3/30: Train loss 1.1870, acc 0.6807 | Val   loss 0.6490, acc 0.7909
Epoch 4/30: Train loss 1.1241, acc 0.7105 | Val   loss 0.6187, acc 0.8061
Epoch 5/30: Train loss 1.0491, acc 0.7134 | Val   loss 0.5781, acc 0.8192
Epoch 6/30: Train loss 0.9714, acc 0.7391 | Val   loss 0.5577, acc 0.8384
Epoch 7/30: Train loss 0.9754, acc 0.7466 | Val   loss 0.5390, acc 0.8515
Epoch 8/30: Train loss 0.9675, acc 0.7402 | Val   loss 0.6355, acc 0.8172
Epoch 9/30: Train loss 0.8921, acc 0.7564 | Val   loss 0.6518, acc 0.8131
Epoch 10/30: Train loss 0.8737, acc 0.7672 | Val   loss 0.5450, acc 0.8545
Epoch 11/30: Train loss 0.9088, acc 0.7551 | Val   loss 0.5490, acc 0.8495
Epoch 12/30: Train loss 0.7599, acc 0.7890 | Val   loss 0.5822, acc 0.8354
Epoch 13/30: Train loss 0.7994, acc 0.7901 | Val   loss 0.4655, acc 0.8707
Epoch 14/30: Tr